# Simple character level tokenizer

In [3]:
with open('training-text.txt', 'r') as f:
    text = f.read()
chars = sorted(list(set(text)))
print("\nVocabulary size is ", len(chars), "chars/tokens: ", ''.join(chars))

ctoi, itoc = dict((char, index) for index, char in enumerate(chars)), dict((index, char) for index, char in enumerate(chars))
print("\nVocabulary: ",  ctoi)

encode = lambda sentence: [ctoi[char] for char in sentence] # encode sentence to index
decode = lambda indexes: ''.join([itoc[index] for index in indexes]) # decode index to sentence
print("example: ",encode("some text for training"), decode(encode("some text for training")))


Vocabulary size is  13 chars/tokens:   aefgimnorstx

Vocabulary:  {' ': 0, 'a': 1, 'e': 2, 'f': 3, 'g': 4, 'i': 5, 'm': 6, 'n': 7, 'o': 8, 'r': 9, 's': 10, 't': 11, 'x': 12}
example:  [10, 8, 6, 2, 0, 11, 2, 12, 11, 0, 3, 8, 9, 0, 11, 9, 1, 5, 7, 5, 7, 4] some text for training


In [4]:
import torch
data = torch.LongTensor(encode(text))
print("\nData shape: ", data.shape, "\nData type: ", data.dtype)


Data shape:  torch.Size([22])
